# 3. Linear Models for Classification

### *Table of Contents*
* 4.1 [Discriminant Functions](#4.1-Discriminant-Functions)
* 4.2 [Probabilistic Generative Models](#4.2-Probabilistic-Generative-Models)
* 4.3 [Probabilistic Discriminative Models](#4.3-Probabilistic-Discriminative-Models)
* 4.4 [Laplace Approximation](#4.4-Laplace-Approximation)

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

# Set random seed to make deterministic
np.random.seed(0)

# Ignore zero divisions and computation involving NaN values.
np.seterr(divide = 'ignore', invalid='ignore')

# Enable higher resolution plots
%config InlineBackend.figure_format = 'retina'